In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
df_meal

In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
len(speech_play.index)

In [ ]:
df_grouped_play

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
speech_meal

In [ ]:
# remove label column from df_grouped_meal 
df_grouped_meal.reset_index()
df_grouped_meal.set_index('s_id', append=True, inplace=True)


In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
meal_data.columns

In [ ]:
meal_data = meal_data.drop(['label'], axis=1)
meal_data

In [ ]:
meal_data.columns

In [ ]:
meal_data = meal_data.dropna()

In [ ]:
df_grouped_play

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
#play_data = play_data.dropna()

In [ ]:
play_data

In [ ]:
len(meal_data['label'])

In [ ]:
speech_child_movement_cols = np.append(speech_meal.columns.values[:-1], ['child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min'])

In [ ]:
play_data

In [ ]:
df_grouped_meal

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_meal[['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = df_grouped_meal['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01]
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
 results_df.loc[results_df['best_score'].idxmax()]

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
# check correlation of cg_movement_mean and child_movement_mean in meal_data
play_data[['cg_movement_mean', 'child_movement_mean']].corr()

In [ ]:
['cg_movement_max', 'cg_movement_min', 'cg_movement_mean', 'cg_movement_var','child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 
 'child_movement_mean', 'child_movement_max', 'child_movement_var', 'child_movement_min', 'cg_movement_mean', 'cg_movement_max', 'cg_movement_var', 'cg_movement_min',
 ]

In [ ]:
play_data

# PCA only for movement features

In [ ]:
meal_data.columns

In [ ]:
meal_data = meal_data.drop(['label'])
meal_data.columns

In [ ]:
meal_data['label'].iloc[:,0]

In [ ]:
# Custom transformer to apply PCA to a subset of features

# Pipeline with PCA applied to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)
    

X = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


y = play_data['label'].iloc[:,0]


remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5, shuffle=False)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled= shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        best_pca_components = grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
            
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df

In [ ]:
best_result = max(results, key=lambda x: x['best_score'])
best_params = best_result['best_params']


In [ ]:
best_params, best_result

## take the best performing model outside of the pipeline and test it and then validate it on the other dataset

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_meal = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y_meal = meal_data['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=0)
remaining_features = [feat for feat in X_meal.columns if feat not in subset_pca]
best_params = {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 1}
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
svc_rbf = SVC(C=best_params['svc_rbf__C'], gamma=best_params['svc_rbf__gamma'], kernel='rbf', probability=True)
# Create the final pipeline
pipeline_meal = Pipeline([
    ('preprocessor', preprocessor),
    ('svc_rbf', svc_rbf)
])

# Fit the pipeline
pipeline_meal.fit(X_meal, y_meal)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_meal, X_meal, y_meal, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset





In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_play = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y_play = play_data['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=0)
remaining_features = [feat for feat in X_play.columns if feat not in subset_pca]
best_params = {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 0.01}
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
svc_rbf = SVC(C=best_params['svc_rbf__C'], gamma=best_params['svc_rbf__gamma'], kernel='rbf', probability=True)
# Create the final pipeline
pipeline_play = Pipeline([
    ('preprocessor', preprocessor),
    ('svc_rbf', svc_rbf)
])

# Fit the pipeline
pipeline_play.fit(X_play, y_play)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_play, X_play, y_play, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset





In [ ]:
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered])

In [ ]:
meal_data_filtered.iloc[:,:-3]

In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(meal_data_filtered.iloc[:,:-3])[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(play_data_filtered.iloc[:,:-3])[:,1]

In [ ]:
accuracy_meal = pipeline_play.score(X_meal, y_meal)
print(f'Accuracy of model trained on play data evaluated on meal data: {accuracy_meal}')

In [ ]:
accuracy_play = pipeline_meal.score(X_play, y_play)
print(f'Accuracy of model trained on play data evaluated on meal data: {accuracy_play}')

In [ ]:
combined_data = combined_data.dropna()
combined_labels = combined_data['label'].iloc[:,0]

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


# Define the base models for voting
base_models = [
    ('play', pipeline_play),
    ('meal', pipeline_meal)
]

gkf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create the voting classifier
voting_ensemble = VotingClassifier(estimators=base_models, voting='soft')

# Evaluate the voting ensemble using cross-validation
f1_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='f1_macro', n_jobs=-1)
precision_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='precision_macro',  n_jobs=-1)
recall_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='recall_macro',  n_jobs=-1)
accuracy_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='accuracy', n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}") 

In [ ]:
len(y_meal), len(y_play)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np


cv = KFold(n_splits=5, shuffle=True, random_state=42)
# Get cross-validated predictions for both models
meal_predictions = cross_val_predict(pipeline_meal, X_meal, y_meal, cv=cv, method='predict_proba')
play_predictions = cross_val_predict(pipeline_play, X_play, y_play, cv=cv, method='predict_proba')

# Align the predictions based on participants
# Assuming the index of X_meal and X_play are participant IDs
meal_predictions_df = pd.DataFrame(meal_predictions, columns=['meal_class_0', 'meal_class_1'], index=X_meal.index)
play_predictions_df = pd.DataFrame(play_predictions, columns=['play_class_0', 'play_class_1'], index=X_play.index)

# Combine the aligned predictions
combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=1)
coombined_labels = pd.concat([y_meal, y_play])

# Define the meta-model
meta_model = StackingClassifier(estimators=[
    ('meal_mlp', pipeline_meal),
    ('play_mlp', pipeline_play)
], final_estimator=LogisticRegression())

# Define the parameter grid for the meta-model
param_grid = {
    'final_estimator__C': [0.01, 0.1, 1, 10, 100],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'saga']
}

# Perform grid search for the meta-model
grid_search = GridSearchCV(meta_model, param_grid, cv=cv, scoring='accuracy')
grid_search.fit(combined_predictions, combined_labels)  # Fit on the original data

# Evaluate the best meta-model
best_meta_model = grid_search.best_estimator_
cross_val_scores = cross_val_score(best_meta_model, combined_predictions, y, cv=cv, scoring='accuracy')


print(f'Best Meta-Model Parameters: {grid_search.best_params_}')
print(f'Cross-Validated Accuracy: {cross_val_scores.mean()}')


# another try with the whole dataset now

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X = combined_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y = combined_data['label'].iloc[:,0]
groups = combined_data.index
gkf = GroupKFold(n_splits=5)

remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=4), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])


# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
  
    

import pandas as pd

# Initialize a list to store the results
results = []
best_models = {}
# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled= shuffle(X, y,groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf,groups=groups_shuffled, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled,groups=groups_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        best_pca_components = grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
            
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)


In [ ]:
results_df.groupby('classifier').mean()

## Best model for meal and speech (1f) :SVC rbf {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 1}

## Best model for play and speech (1f) : SVC rbf {'preprocessor__pca_pipeline__n_components': 4, 'svc_rbf__C': 100, 'svc_rbf__gamma': 0.01}

In [ ]:
meal_data = meal_data.dropna()  
play_data = play_data.dropna()  

In [ ]:
# Assuming 'participant_id' is the index in both DataFrames
# Select the relevant columns from both DataFrames
meal_data_selected = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group','label']]
play_data_selected = play_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 'group', 'label']]

# Find the common indices
common_indices = meal_data_selected.index.intersection(play_data_selected.index)

# Filter the DataFrames to keep only the common indices
meal_data_filtered = meal_data_selected.loc[common_indices]
play_data_filtered = play_data_selected.loc[common_indices]

# Concatenate the filtered DataFrames
combined_data = pd.concat([meal_data_filtered, play_data_filtered])

In [ ]:
combined_labels = combined_data['label'].iloc[:,0]

In [ ]:
combined_data = combined_data.dropna()

In [ ]:
len(combined_data[combined_data['group'] == 'meal']), len(combined_data[combined_data['group'] == 'play'])

## the best model for play dataset

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_meal = meal_data[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y = meal_data_filtered['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=42)
remaining_features = [feat for feat in X_meal.columns if feat not in subset_pca]
best_params = {'mlp__activation': 'relu', 'mlp__alpha': 0.01, 'mlp__hidden_layer_sizes': (6,),'mlp__solver':'lbfgs', 'pca__n_components': 4}

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4))
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
mlp = MLPClassifier(activation=best_params['mlp__activation'], alpha=best_params['mlp__alpha'], hidden_layer_sizes=best_params['mlp__hidden_layer_sizes'],
                                                                                                                               solver=best_params['mlp__solver'])

# Create the final pipeline
pipeline_meal = Pipeline([
    ('preprocessor', preprocessor),
    ('mlp', mlp)
])

# Fit the pipeline
pipeline_meal.fit(X_meal, y)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_meal, X_meal, y, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset
predictions_meal = pipeline_meal.predict(X_meal)




In [ ]:
X_transformed = preprocessor.transform(X_meal)

# Extract the PCA component
pca_component = preprocessor.named_transformers_['pca_pipeline'].named_steps['pca']

# Print the explained variance ratio
print(f'Explained variance ratio of PCA components: {pca_component.explained_variance_ratio_}')

# Print the transformed data
print(f'Transformed data (first 5 rows):\n{X_transformed[:5]}')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_play = play_data_filtered[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y = play_data_filtered['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=42)
remaining_features = [feat for feat in X_play.columns if feat not in subset_pca]
best_params = {'mlp__activation': 'tanh', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 2}

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4))
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters
mlp = MLPClassifier(activation=best_params['mlp__activation'], alpha=best_params['mlp__alpha'], hidden_layer_sizes=best_params['mlp__hidden_layer_sizes'])

# Create the final pipeline
pipeline_play = Pipeline([
    ('preprocessor', preprocessor),
    ('mlp', mlp)
])

# Fit the pipeline
pipeline_play.fit(X_play, y)

# Evaluate the model using cross-validation
scores = cross_val_score(pipeline_play, X_play, y, scoring = 'accuracy', cv=cv)
print(f'Cross-validation scores: {scores}')
print(f'Mean cross-validation score: {np.mean(scores)}')

# Make predictions on the dataset
predictions_play = pipeline_play.predict(X)

# Calculate accuracy, F1 score, precision, and recall
accuracy = accuracy_score(y, predictions)
f1 = f1_score(y, predictions, average='weighted')
precision = precision_score(y, predictions, average='weighted')
recall = recall_score(y, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')



In [ ]:
## correlations between the two models 
# Get predictions from the LR model
meal_predictions =  pipeline_meal.predict_proba(X)[:,1]

# Get predictions from the MLP model
play_predictions = pipeline_play.predict_proba(X)[:,1]

In [ ]:
# Compute the correlation between the predictions
from scipy.stats import pearsonr
correlation, p_value = pearsonr(meal_predictions, play_predictions)

print(f"Correlation between LR and MLP model predictions: {correlation}")
print(f"P-value: {p_value}")

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Calculate Cohen's Kappa
kappa_score = cohen_kappa_score(predictions_meal, predictions_play)
print("Cohen's Kappa score:", kappa_score)

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import confusion_matrix

# Create a contingency table
contingency_table = confusion_matrix(predictions_play, predictions_meal)
result = mcnemar(contingency_table)
print("McNemar's test p-value:", result.pvalue)

In [ ]:

agreement_rate = np.mean(predictions_play == predictions_meal)
print("Prediction Agreement Rate:", agreement_rate)

In [ ]:
combined_data.iloc[:,:-3], combined_labels

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)

# Example data
X_meal = pd.DataFrame(meal_data_filtered[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']])

X_play = pd.DataFrame(play_data_filtered[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']])

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']

y = play_data_filtered['label'].iloc[:,0]

cv = KFold(n_splits=5, shuffle=True, random_state=42)
remaining_features = [feat for feat in X_meal.columns if feat not in subset_pca]
best_params = {'mlp__activation': 'tanh', 'mlp__alpha': 0.001, 'mlp__hidden_layer_sizes': (50, 50), 'pca__n_components': 4}

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=4))
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Define the MLPClassifier with specified hyperparameters for meal model
mlp_meal = MLPClassifier(activation=best_params['mlp__activation'], alpha=best_params['mlp__alpha'], hidden_layer_sizes=best_params['mlp__hidden_layer_sizes'])

# Define the MLPClassifier with specified hyperparameters for play model
mlp_play = MLPClassifier(activation=best_params['mlp__activation'], alpha=best_params['mlp__alpha'], hidden_layer_sizes=best_params['mlp__hidden_layer_sizes'])

# Create the pipeline for meal model
pipeline_meal = Pipeline([
    ('preprocessor', preprocessor),
    ('mlp', mlp_meal)
])

# Create the pipeline for play model
pipeline_play = Pipeline([
    ('preprocessor', preprocessor),
    ('mlp', mlp_play)
])

# Fit the pipelines
pipeline_meal.fit(X_meal, y)
pipeline_play.fit(X_play, y)

# Get cross-validated predictions for both models
meal_predictions = cross_val_predict(pipeline_meal, X_meal, y, cv=cv, method='predict_proba')
play_predictions = cross_val_predict(pipeline_play, X_play, y, cv=cv, method='predict_proba')

# Align the predictions based on participants
# Assuming the index of X_meal and X_play are participant IDs
meal_predictions_df = pd.DataFrame(meal_predictions, columns=['meal_class_0', 'meal_class_1'], index=X_meal.index)
play_predictions_df = pd.DataFrame(play_predictions, columns=['play_class_0', 'play_class_1'], index=X_play.index)

# Combine the aligned predictions
combined_predictions = pd.concat([meal_predictions_df, play_predictions_df], axis=1)


# Define the meta-model
meta_model = StackingClassifier(estimators=[
    ('meal_mlp', pipeline_meal),
    ('play_mlp', pipeline_play)
], final_estimator=LogisticRegression())

# Define the parameter grid for the meta-model
param_grid = {
    'final_estimator__C': [0.01, 0.1, 1, 10, 100],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'saga']
}

# Perform grid search for the meta-model
grid_search = GridSearchCV(meta_model, param_grid, cv=cv, scoring='accuracy')
grid_search.fit(combined_predictions, y)  # Fit on the original data

# Evaluate the best meta-model
best_meta_model = grid_search.best_estimator_
cross_val_scores = cross_val_score(best_meta_model, combined_predictions, y, cv=cv, scoring='accuracy')


print(f'Best Meta-Model Parameters: {grid_search.best_params_}')
print(f'Cross-Validated Accuracy: {cross_val_scores.mean()}')


In [ ]:
# Get cross-validated predictions for both models
meal_predictions = cross_val_predict(pipeline_meal, X_meal, y, cv=cv, method='predict_proba')
play_predictions = cross_val_predict(pipeline_play, X_play, y, cv=cv, method='predict_proba')

# Check the shape of predictions
print("Meal Predictions Shape:", meal_predictions.shape)  # Should be (n_samples, n_classes)
print("Play Predictions Shape:", play_predictions.shape)  # Should be (n_samples, n_classes)

# Combine predictions into a single DataFrame
combined_predictions = pd.DataFrame(
    np.hstack((meal_predictions, play_predictions)),
    columns=['meal_class_0', 'meal_class_1', 'play_class_0', 'play_class_1'],
    index=X_meal.index  # Ensure index aligns with target 'y'
)

# Verify combined predictions
print("Combined Predictions Shape:", combined_predictions.shape)  # Should be (n_samples, 4)
print("Combined Predictions Columns:", combined_predictions.columns)  # Check column names

# Meta-model with StackingClassifier
meta_model = StackingClassifier(estimators=[
    ('meal_mlp', pipeline_meal),
    ('play_mlp', pipeline_play)
], final_estimator=LogisticRegression())

# Fit the meta-model
meta_model.fit(combined_predictions, y)

# Evaluate the meta-model
cross_val_scores = cross_val_score(meta_model, combined_predictions, y, cv=cv, scoring='accuracy')

print(f'Cross-Validated Accuracy: {cross_val_scores.mean()}')

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier


# Define the base models for voting
base_models = [
    ('play', pipeline_play),
    ('meal', pipeline_meal)
]

gkf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create the voting classifier
voting_ensemble = VotingClassifier(estimators=base_models, voting='soft')

# Evaluate the voting ensemble using cross-validation
f1_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='f1_macro', n_jobs=-1)
precision_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='precision_macro',  n_jobs=-1)
recall_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='recall_macro',  n_jobs=-1)
accuracy_scores = cross_val_score(voting_ensemble, combined_data, combined_labels, cv=gkf, scoring='accuracy', n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

In [ ]:
weights = [0.5, 0.5]  # Adjust these weights based on model performance

# Initialize lists to store the evaluation metrics
f1_scores = []
precision_scores = []
recall_scores = []
accuracy_scores = []
gkf = KFold(n_splits=5, shuffle=True, random_state=42)
# Perform manual cross-validation
for train_index, test_index in gkf.split(combined_data, combined_labels):
    X_train, X_test = combined_data.iloc[train_index], combined_data.iloc[test_index]
    y_train, y_test = combined_labels.iloc[train_index], combined_labels.iloc[test_index]
    
    # Fit the base models
    for name, model in base_models:
        model.fit(X_train, y_train)
    
    # Get the predicted probabilities from each base model
    meal_probs = pipeline_meal.predict_proba(X_test)[:, 1]
    play_probs = pipeline_play.predict_proba(X_test)[:, 1]
    
    # Combine the predicted probabilities using weighted averaging
    combined_probs = (weights[0] * meal_probs) + (weights[1] * play_probs)
    
    # Convert combined probabilities to class predictions
    combined_preds = np.where(combined_probs > 0.5, 1, 0)
    
    # Calculate evaluation metrics
    f1_scores.append(f1_score(y_test, combined_preds, average='macro'))
    precision_scores.append(precision_score(y_test, combined_preds, average='macro'))
    recall_scores.append(recall_score(y_test, combined_preds, average='macro'))
    accuracy_scores.append(accuracy_score(y_test, combined_preds))

# Print the evaluation results
print(f"F1 Score: {np.mean(f1_scores)}")
print(f"Precision: {np.mean(precision_scores)}")
print(f"Recall: {np.mean(recall_scores)}")
print(f"Accuracy: {np.mean(accuracy_scores)}")

In [ ]:
# Define the meta-model
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Evaluate the meta-model using cross-validation
f1_scores = cross_val_score(meta_model, combined_predictions, y, cv=gkf, scoring='f1_macro', n_jobs=-1)
precision_scores = cross_val_score(meta_model, combined_predictions, y, cv=gkf, scoring='precision_macro',  n_jobs=-1)
recall_scores = cross_val_score(meta_model, combined_predictions, y, cv=gkf, scoring='recall_macro',  n_jobs=-1)
accuracy_scores = cross_val_score(meta_model, combined_predictions, y, cv=gkf, scoring='accuracy', n_jobs=-1)

# Print the evaluation results
print(f"F1 Score: {f1_scores.mean()}")
print(f"Precision: {precision_scores.mean()}")
print(f"Recall: {recall_scores.mean()}")
print(f"Accuracy: {accuracy_scores.mean()}")

In [ ]:
combined_data